In [1]:
import pandas as pd
import ast
from collections import defaultdict as dd
import regex as re
import seaborn as sns
import numpy as np

# Descriptive Analytics

In [2]:
df = pd.read_csv("sample.txt", header=0, index_col = 0)
df.head()

/home/isaac/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event,Site,GameNumber,WName,BName,WElo,BElo,WRD,BRD,WIsComp,...,TimeControl,Date,Time,WClock,BClock,ECO,PlyCount,Result,WMoves,BMoves
0,FICS rated blitz game,FICS freechess.org,470452425,chesspickle,lawrencegern,1963,764,21.2,112.2,Yes,...,300+0,2020.01.31,23:54:00,05:00.0,05:00.0,D11,43,1-0,"['d4', 'c4', 'Nf3', 'e3', 'Bxc4', 'Be2', 'Bd2'...","['d5', 'c6', 'dxc4', 'e6', 'b5', 'Bb4+', 'Bxd2..."
1,FICS rated blitz game,FICS freechess.org,470452424,ZoneBlindTal,AhNeow,1600,1487,36.6,28.5,NaN,...,180+2,2020.01.31,23:52:00,03:00.0,03:00.0,C41,48,0-1,"['e4', 'Nf3', 'd4', 'dxe5', 'Nc3', 'Nd5', 'Bd2...","['e5', 'd6', 'Qe7', 'dxe5', 'f6', 'Qd6', 'c6',..."
2,FICS rated blitz game,FICS freechess.org,470452423,forlat,naraindra,1948,1089,15.3,30.7,Yes,...,300+0,2020.01.31,23:54:00,05:00.0,05:00.0,D00,67,1-0,"['d4', 'g3', 'Bg2', 'Nf3', 'O-O', 'e3', 'c3', ...","['d5', 'b5', 'Bb7', 'h6', 'e6', 'Bd6', 'Nf6', ..."
3,FICS rated blitz game,FICS freechess.org,470452420,PGierech,kkpsA,1497,1224,32.4,37.6,NaN,...,120+12,2020.01.31,23:55:00,02:00.0,02:00.0,D00,23,1-0,"['e4', 'd4', 'dxe5', 'exd5', 'Nf3', 'Be2', 'Bx...","['d5', 'e5', 'Nc6', 'Nxe5', 'Bg4', 'Nxf3+', 'B..."
4,FICS rated blitz game,FICS freechess.org,470452419,Liszt,Sotavaunu,1594,1554,26.8,34.9,NaN,...,300+2,2020.01.31,23:48:00,05:00.0,05:00.0,B06,53,0-1,"['e4', 'd4', 'f4', 'c3', 'cxd4', 'e5', 'Nc3', ...","['g6', 'Bg7', 'c5', 'cxd4', 'd5', 'Nc6', 'Nh6'..."


In [3]:
df["BMoves"] = df["BMoves"].apply(ast.literal_eval)
df["WMoves"] = df["WMoves"].apply(ast.literal_eval)

## Most popular square?

In [4]:
lookups = [
    ("^.*[a-h][1-8].*$", "normal move"),
    ("^O-O$", "kingside castle"),
    ("^O-O-O$", "queenside castle"),
    ("^.+[QNBR]\+*#*$", "pawn conversion"),
    ("^.*\+$", "check"),
    ("^.*#$", "checkmate")
]

def is_not_castle(move):
    return not bool(re.search("O-O",move))


In [5]:
square_dict = dd(int)

def count_squares(moves, colour):
    for move in moves:
        if is_not_castle(move):
            square_dict[re.findall("[a-h][1-8]",move)[-1]] += 1
        else:
            if re.search("^O-O$", move):
                if colour=="W":
                    square_dict["g1"] += 1
                else:
                    square_dict["g8"] += 1
            else: 
                if colour=="W":
                    square_dict["c1"] += 1
                else:
                    square_dict["c8"] += 1
    return


In [34]:
df["BMoves"].apply(lambda x : count_squares(x,'B'))
df["WMoves"].apply(lambda x : count_squares(x,'W'))

0        None
1        None
2        None
3        None
4        None
         ... 
39255    None
39256    None
39257    None
39258    None
39259    None
Name: WMoves, Length: 39260, dtype: object

## How many moves until a game becomes unique?

In [ ]:
moves_until_unique = dd(int)

for i in range(len(sample_df["WMoves"])):
    for j in range(i,len(sample_df["WMoves"])):
        move_count = -1
        g1move = ""
        g2move = ""
        while g1move==g2move:
            move_count += 1
            try:
                if move_count%2 == 0:
                    #White's move
                    g1move = df["WMoves"][i][move_count//2]
                    g2move = df["WMoves"][j][move_count//2]
                else:
                    #Black's move
                    g1move = df["BMoves"][i][move_count//2]
                    g2move = df["BMoves"][j][move_count//2]
            except:
                #end of game
                g1move = 1
                g2move = 2
        if moves_until_unique[i] < move_count:
           moves_until_unique[i] = move_count
        if moves_until_unique[j] < move_count:
           moves_until_unique[j] = move_count
            

In [ ]:
moves_until_unique

In [ ]:
df.loc[195]

In [ ]:
sample_df = df.head(1000)

# Prescriptive Analytics

## Feature Engineering

## Add columns for who won game

In [ ]:
def get_player_result(result, colour):
    if colour=='B':
        if result[-1]=='0':
            return 0
        else:
            return 1
    else:
        if result[0]=='0':
            return 0
        else:
            return 1

def add_WL_columns(df):
    df['WResult'] = df["Result"].apply(lambda x: get_player_result(x, 'W'))
    df['BResult'] = df["Result"].apply(lambda x: get_player_result(x, 'B'))
    return

## Add columns for what players converted pawns to

In [ ]:
def get_conversions(game, colour):
    conversions = []
    for i in range(len(game)):
        if i%2 and colour=='B':
            if re.match(lookups[3][0], game[i]):
                conversions.append(re.search("(?<=[a-h][1-8])[QNBR]",game[i]).group(0))
        else:
            if re.match(lookups[3][0], game[i]):
                conversions.append(re.search("(?<=[a-h][1-8])[QNBR]",game[i]).group(0))
    return conversions

def add_conversions_columns(df):
    df["WConversions"] = df["game"].apply(lambda x: get_conversions(x, 'W'))
    df["BConversions"] = df["game"].apply(lambda x: get_conversions(x, 'B'))